In [2]:
from transformers import AutoModelForSeq2SeqLM
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType
model_name_or_path = "bigscience/mt0-large"
tokenizer_name_or_path = "bigscience/mt0-large"

peft_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1
)

model = AutoModelForSeq2SeqLM.from_pretrained(model_name_or_path)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()
"trainable params: 2359296 || all params: 1231940608 || trainable%: 0.19151053100118282"

trainable params: 2,359,296 || all params: 1,231,940,608 || trainable%: 0.1915


'trainable params: 2359296 || all params: 1231940608 || trainable%: 0.19151053100118282'

In [3]:
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer
import torch

model = AutoPeftModelForCausalLM.from_pretrained("ybelkada/opt-350m-lora").to("cuda")
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-350m")

model.eval()
inputs = tokenizer("how is datascince jobs in future", return_tensors="pt")

outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=50)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True)[0])

"Preheat the oven to 350 degrees and place the cookie dough in the center of the oven. In a large bowl, combine the flour, baking powder, baking soda, salt, and cinnamon. In a separate bowl, combine the egg yolks, sugar, and vanilla."

adapter_config.json:   0%|          | 0.00/416 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/644 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/663M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/662M [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

how is datascince jobs in future?
I think it's a good thing. I'm not sure if it's a good thing or not, but it's a good thing.


'Preheat the oven to 350 degrees and place the cookie dough in the center of the oven. In a large bowl, combine the flour, baking powder, baking soda, salt, and cinnamon. In a separate bowl, combine the egg yolks, sugar, and vanilla.'

In [3]:
# pip install trl
!git clone https://github.com/huggingface/trl

!python trl/examples/scripts/sft.py \
    --model_name meta-llama/Llama-2-7b-hf \
    --dataset_name timdettmers/openassistant-guanaco \
    --load_in_4bit \
    --use_peft \
    --batch_size 4 \
    --gradient_accumulation_steps 2 \
    --log_with wandb

Cloning into 'trl'...
remote: Enumerating objects: 18868, done.
remote: Counting objects: 100% (1207/1207), done.
remote: Compressing objects: 100% (633/633), done.
remote: Total 18868 (delta 995), reused 574 (delta 574), pack-reused 17661 (from 5)
Receiving objects: 100% (18868/18868), 13.06 MiB | 28.88 MiB/s, done.
Resolving deltas: 100% (14016/14016), done.


In [1]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    pipeline,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

model_name = "meta-llama/Llama-2-7b-chat-hf"
new_model = "manojpatil/llama-2-7b-chat-lora-adaptor"
device_map = {"": 0}

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

prompt = "Who draw the picture of Monalisa"

pipe = pipeline(task="text-generation", model=base_model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_config.json:   0%|          | 0.00/453 [00:00<?, ?B/s]

adapter_model.bin:   0%|          | 0.00/134M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Device set to use cuda:0
Device set to use cuda:0


<s>[INST] Who draw the picture of Monalisa [/INST]  The painting of the Mona Lisa is attributed to the Italian artist Leonardo da Vinci (1452-1519). The painting is a portrait of a woman who is believed to be Lisa Gherardini, the wife of a wealthy merchant named Francesco del Giocondo. (The name "Mona Lisa" is not a reference to the subject's smile, but rather to the Italian word for "Madame.")

Leonardo da Vinci began working on the Mona Lisa in 1503 and completed it around 1506. The painting is oil on poplar wood and is relatively small, measuring only 77 x 53 cm (30 x 21 inches). It is now housed in the Louvre Museum in Paris, France, where it is considered one of the most famous paint
<s>[INST] Who draw the picture of Monalisa [/INST]  The Mona Lisa painting is attributed to Leonardo da Vinci, an Italian artist and polymath, who lived from 1452 to 1519.

Leonardo da Vinci was a true Renaissance man, and his artistic talents were not limited to painting. He was also a sculptor, arch

In [2]:
# !pip install --upgrade transformers
# !pip install huggingface_hub

from huggingface_hub import login
login()

In [2]:
import transformers
import torch

model_id = "meta-llama/Llama-3.1-8B"

pipeline = transformers.pipeline(
    "text-generation", model=model_id, model_kwargs={"torch_dtype": torch.bfloat16}, device_map="auto"
)

pipeline("Hey how is the weather today in hyderabad india?")

config.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!
`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': 'Hey how is the weather today in hyderabad india??\ni am from hyderabad and i am enjoying a rainy day here.\ni love the weather here, its cold and the rain is just great.'}]

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from PIL import Image
import requests
from huggingface_hub import login # Import login function
from google.colab import userdata # Import userdata to access secrets

# Get your Hugging Face token from Colab Secrets Manager
# Save your token to Colab Secrets Manager under the name 'HF_TOKEN'
huggingface_token = userdata.get("HF_TOKEN")

if huggingface_token is None:
    print("Please add your Hugging Face token to Colab Secrets Manager with the name 'HF_TOKEN'")
else:
    login(token=huggingface_token) # Login to Hugging Face

    # Check if GPU is available
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # Load the tokenizer and model
    model_name = "meta-llama/Llama-3.2-11B-Vision"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

    # Prepare your input prompt for text generation
    text_prompt = "What do you see in this image?"

    # Load and preprocess an image
    image_url = "https://fileinfo.com/img/ss/xl/jpg_44-2.jpg"  # Replace with your image URL
    image = Image.open(requests.get(image_url, stream=True).raw)

    # Here, you would typically preprocess the image as required by the model
    # (specific preprocessing steps may depend on the model's requirements)

    # Generate a response
    inputs = tokenizer(text_prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        output = model.generate(inputs['input_ids'], max_length=100)

    # Decode and print the output
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    print("Generated Text:", generated_text)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
import json
import torch
from transformers import (AutoTokenizer, AutoModelForCausalLM,BitsAndBytesConfig,pipeline)

In [ ]:
config_data = json.load(open('config.json'))
HF_TOKEN = config_data['HF_TOKEN']